In [15]:
import numpy as np, pandas as pd

In [2]:
dataset_name = "ionosphere"

data_filename = "../data/{}0.csv".format(dataset_name)

data_raw = pd.read_csv(data_filename)

class_prepared = pd.get_dummies(data_raw.iloc[:,-1])
data_mod = data_raw.iloc[:,:-1]

data_mod.to_csv("../data/{}_train.tab".format(dataset_name), sep=" ", index=False)
class_prepared.to_csv("../data/{}_train.Y".format(dataset_name), sep=" ", index=False)

In [3]:
%run ../apyori.py

# below function basically takes a data frame and a support threshold and returns itemsets which satisfy the threshold
def run_apriori(df, support_thres):
    # the idea is to basically make a list of strings out of df and run apriori api on it 
    # return the frequent itemsets
    dataset = []
    for i in range(0,df.shape[0]):
        temp = []
        for col_name in df.columns:
            temp.append(col_name+"="+str(df[col_name][i]))
        dataset.append(temp)

    results = list(apriori(dataset, min_support=support_thres))
    
    list_itemsets = []
    for ele in results:
        temp = []
        for pred in ele.items:
            temp.append(pred)
        list_itemsets.append(temp)

    return list_itemsets

In [7]:
import fim

def process_dataset(df):
    dataset = []
    for i in range(0,df.shape[0]):
        temp = []
        for col_name in df.columns:
            temp.append(col_name+"="+str(df[col_name][i]))
        dataset.append(temp)
        
    return dataset
    

def run_fim_apriori(df, support_thres):
    # the idea is to basically make a list of strings out of df and run apriori api on it 
    # return the frequent itemsets
    dataset = []
    for i in range(0,df.shape[0]):
        temp = []
        for col_name in df.columns:
            temp.append(col_name+"="+str(df[col_name][i]))
        dataset.append(temp)
        
    print("dataset processed")

    result_raw = fim.apriori(dataset, supp=support_thres)
    result = list(map(lambda i: list(i[0]), result_raw))
        
    return result

In [8]:
df = pd.read_csv('../titanic_train.tab',' ', header=None, names=['Passenger_Cat', 'Age_Cat', 'Gender'])
df1 = pd.read_csv('../titanic_train.Y', ' ', header=None, names=['Died', 'Survived'])
Y = list(df1['Died'].values)

itemsets = run_apriori(df, 0.1)

itemsets

[['Age_Cat=adult'],
 ['Gender=female'],
 ['Gender=male'],
 ['Passenger_Cat=1st_class'],
 ['Passenger_Cat=2nd_class'],
 ['Passenger_Cat=3rd_class'],
 ['Passenger_Cat=crew'],
 ['Gender=female', 'Age_Cat=adult'],
 ['Gender=male', 'Age_Cat=adult'],
 ['Passenger_Cat=1st_class', 'Age_Cat=adult'],
 ['Passenger_Cat=2nd_class', 'Age_Cat=adult'],
 ['Age_Cat=adult', 'Passenger_Cat=3rd_class'],
 ['Passenger_Cat=crew', 'Age_Cat=adult'],
 ['Gender=male', 'Passenger_Cat=3rd_class'],
 ['Gender=male', 'Passenger_Cat=crew'],
 ['Gender=male', 'Age_Cat=adult', 'Passenger_Cat=3rd_class'],
 ['Gender=male', 'Passenger_Cat=crew', 'Age_Cat=adult']]

In [9]:
df = pd.read_csv('../data/ionosphere0.csv')

df_raw = df.iloc[:, :-1]
Y = df.iloc[:, -1]

def run_fim_apriori(df, minsup):
    processed_df = process_dataset(df)
    
    result_raw = fim.apriori(processed_df, supp=(minsup*100))
    result = list(map(lambda i: list(i[0]), result_raw))
    
    return result


rules = run_fim_apriori(df_raw, 0.8)

len(rules)


200

In [10]:
%run ../IDS_smooth_local.py

list_of_rules = createrules(rules, list(set(Y)))

len(list_of_rules)

400

In [17]:


#!more "../train_model_SLS.py"



print("----------list of rules------------")

print("-------------rules created---------")
for r in list_of_rules:
    r.print_rule()

lambda_array = [1.0]*7     # use separate hyperparamter search routine
s1 = smooth_local_search(list_of_rules, df, Y, lambda_array, 0.33, 0.33)
s2 = smooth_local_search(list_of_rules, df, Y, lambda_array, 0.33, -1.0)
f1 = func_evaluation(s1, list_of_rules, df, Y, lambda_array)
f2 = func_evaluation(s2, list_of_rules, df, Y, lambda_array)

result_set = {}
if f1 > f2:
    print("The Solution Set is: "+str(s1))
    result_set = list(s1)
else:
    print("The Solution Set is: "+str(s2))
    result_set = list(s2)

----------list of rules------------
-------------rules created---------
If sepalwidth == 3.35_to_inf, then Iris-versicolor
If sepalwidth == 3.35_to_inf, then Iris-setosa
If sepalwidth == 3.35_to_inf, then Iris-virginica
If petallength == -inf_to_2.45 and petalwidth == -inf_to_0.8 and sepalwidth == 3.35_to_inf, then Iris-versicolor
If petallength == -inf_to_2.45 and petalwidth == -inf_to_0.8 and sepalwidth == 3.35_to_inf, then Iris-setosa
If petallength == -inf_to_2.45 and petalwidth == -inf_to_0.8 and sepalwidth == 3.35_to_inf, then Iris-virginica
If petalwidth == -inf_to_0.8 and sepalwidth == 3.35_to_inf, then Iris-versicolor
If petalwidth == -inf_to_0.8 and sepalwidth == 3.35_to_inf, then Iris-setosa
If petalwidth == -inf_to_0.8 and sepalwidth == 3.35_to_inf, then Iris-virginica
If petallength == -inf_to_2.45 and petalwidth == -inf_to_0.8 and sepallength == -inf_to_5.55 and sepalwidth == 3.35_to_inf, then Iris-versicolor
If petallength == -inf_to_2.45 and petalwidth == -inf_to_0.8 an

TypeError: 'NoneType' object is not iterable

In [11]:
for rule in list_of_rules:
        rule.cover = rule.get_cover(df)
        rule.correct_cover = rule.get_correct_cover(df, Y)

for r1 in list_of_rules:
        for r2 in list_of_rules:
            OVERLAP_CACHE[frozenset([r1, r2])] = overlap(r1, r2, df)

In [12]:
overlap(list_of_rules[0], list_of_rules[1], df)

[0,
 1,
 2,
 4,
 5,
 6,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 18,
 19,
 21,
 23,
 28,
 29,
 30,
 31,
 32,
 34,
 35,
 36,
 38,
 40,
 42,
 43,
 44,
 45,
 49,
 51,
 53,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 63,
 64,
 65,
 67,
 68,
 69,
 71,
 75,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 90,
 91,
 93,
 95,
 96,
 97,
 98,
 99,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 114,
 115,
 116,
 117,
 118,
 119,
 121,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 147,
 149,
 151,
 152,
 153,
 155,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 165,
 167,
 168,
 174,
 175,
 176,
 178,
 179,
 180,
 181,
 182,
 184,
 185,
 186,
 187,
 188,
 190,
 191,
 192,
 194,
 196,
 198,
 200,
 202,
 204,
 205,
 207,
 209,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 219,
 221,
 223,
 226,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 242,

In [13]:
def optimize():

        
    """
    for r1 in list_of_rules:
        for r2 in list_of_rules:
            OVERLAP_CACHE[repr(r1) + repr(r2)] = overlap(r1, r2, df)
    """
    
    print("here")
    
    lambda_array = [1.0]*7     # use separate hyperparamter search routine
    s1 = smooth_local_search(list_of_rules, df, Y, lambda_array, 0.33, 0.33)
    s2 = smooth_local_search(list_of_rules, df, Y, lambda_array, 0.33, -1.0)
    f1 = func_evaluation(s1, list_of_rules, df, Y, lambda_array)
    f2 = func_evaluation(s2, list_of_rules, df, Y, lambda_array)

    result_set = {}
    if f1 > f2:
        print("The Solution Set is: "+str(s1))
        result_set = list(s1)
    else:
        print("The Solution Set is: "+str(s2))
        result_set = list(s2)

In [21]:
r1 = list_of_rules[0]
r2 = list_of_rules[1]
frozenset([r1, r2]) == frozenset([r2, r1])


{frozenset([r1, r2]): 5}

{frozenset({<__main__.rule at 0x1133010f400>,
            <__main__.rule at 0x1133010fb00>}): 5}

In [14]:
import cProfile

cProfile.run("optimize()")

here
2/n*n OPT value is 1199.0393375
Estimating omega for rule 0
Standard Error 168242.26994429788
Standard Error 101881.20989238827
Standard Error 77516.79873278317
Standard Error 70248.14709622807
Standard Error 65061.58190905333
Standard Error 58427.85520933673
Standard Error 54090.834346118725
Standard Error 48722.13546417502
Standard Error 45890.92163365245
Standard Error 43187.698319204406
Standard Error 41653.40147872082
Standard Error 40491.50532226069
Standard Error 38662.41388400214
Standard Error 36651.04153951278
Standard Error 34869.60061280273
Standard Error 34140.18863672455
Standard Error 32570.58257807747
Standard Error 31456.448544922634
Standard Error 30773.167335419093
Standard Error 29982.590679409263
Standard Error 29398.93838328602
Standard Error 28575.482108254866
Standard Error 27900.83076355199
Standard Error 27553.289972845618
Standard Error 27335.93193329202
Standard Error 26770.561848063157
Standard Error 26083.734064920827
Standard Error 25712.053120800203

KeyboardInterrupt: 

In [55]:
np_rules = np.array(list_of_rules)
solution_rules = np_rules[result_set]

list(map(lambda r: r.print_rule(), solution_rules))

If petalwidth == 0.8_to_1.75, then Iris-virginica
If petalwidth == 0.8_to_1.75, then Iris-versicolor
If petallength == 4.75_to_inf, then Iris-setosa
If sepalwidth == 2.95_to_3.35, then Iris-virginica
If sepalwidth == 2.95_to_3.35, then Iris-versicolor
If sepallength == -inf_to_5.55, then Iris-virginica


[None, None, None, None, None, None]

In [60]:
r = np_rules[0]


set(r.itemset) 

{('petalwidth', '0.8_to_1.75')}

In [87]:
def prepare_dataset_to_tuples(df):
    df_dict = df.iloc[:, :-1].to_dict(orient="records")
    df_items = list(map(lambda i: set(i.items()), df_dict))

    return df_items


def predict(rule_list, instance):
    matches = []
    
    for rule in rule_list:
        if r.itemset <= instance:
            matches.append(r)
            
    return matches


df_items = prepare_dataset_to_tuples(df)


predict(solution_rules, df_items[3])

print(df_items[1])

list(map(lambda r: r.print_rule(), solution_rules))

{('petallength', '-inf_to_2.45'), ('sepalwidth', '2.95_to_3.35'), ('petalwidth', '-inf_to_0.8'), ('sepallength', '-inf_to_5.55')}
If petalwidth == 0.8_to_1.75, then Iris-virginica
If petalwidth == 0.8_to_1.75, then Iris-versicolor
If petallength == 4.75_to_inf, then Iris-setosa
If sepalwidth == 2.95_to_3.35, then Iris-virginica
If sepalwidth == 2.95_to_3.35, then Iris-versicolor
If sepallength == -inf_to_5.55, then Iris-virginica


[None, None, None, None, None, None]

In [145]:
def get_cover(r, df):
    dfnew = df.copy()
    for pattern in r.itemset: 
        dfnew = dfnew[dfnew[pattern[0]] == pattern[1]]
    return list(dfnew.index.values)

def get_cover_faster(r, df):
    mask = np.ones(len(df), dtype=bool)
    for pattern in r.itemset:
        mask &= df[pattern[0]] == pattern[1]
        
    return list(df[mask].index.values)


cache = {}
def get_cover_memoized(r, df):    
    result = cache.get(repr(r))
    
    if result:
        return result
    
    mask = np.ones(len(df), dtype=bool)
    for pattern in r.itemset:
        mask &= df[pattern[0]] == pattern[1]
        
    result = list(df[mask].index.values)
    
    cache[repr(r)] = result 
        
    return result
    
df.copy()

#%timeit get_cover(r, df)
#%timeit get_cover_faster(r, df)


get_cover_memoized(r, df)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 13,
 14,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 47,
 51,
 53,
 54,
 71,
 74,
 79,
 80,
 83,
 88,
 96]

In [130]:
d = {"1": 2}

d.get("1")

2

In [120]:
faster_ratio = 1.03 / 1.3

faster_ratio * 1.3

39.067 * faster_ratio

30.953084615384615

In [12]:
def get_correct_cover(r, df, Y):
    indexes_points_covered = r.get_cover(df) # indices of all points satisfying the rule
    Y_arr = pd.Series(Y)                    # make a series of all Y labels
    labels_covered_points = list(Y_arr[indexes_points_covered])   # get a list only of Y labels of the points covered
    correct_cover = []
    for ind in range(0,len(labels_covered_points)):
        if labels_covered_points[ind] == r.class_label:
            correct_cover.append(indexes_points_covered[ind])
    return correct_cover, indexes_points_covered


def get_correct_cover_faster(r, df, Y):
    indexes_points_covered = r.get_cover(df) # indices of all points satisfying the rule
    Y_arr = np.array(Y)                    # make a series of all Y labels
    labels_covered_points = Y_arr[indexes_points_covered]   # get a list only of Y labels of th
    mask = labels_covered_points == r.class_label
    result = np.array(indexes_points_covered)[mask]

    return result, indexes_points_covered


CORRECT_COVER_CACHE = {}

def get_correct_cover_memoized(r, df, Y):
    result = CORRECT_COVER_CACHE.get(repr(r))
    
    if result:
        return result
    
    indexes_points_covered = r.get_cover(df) # indices of all points satisfying the rule
    Y_arr = np.array(Y)                    # make a series of all Y labels
    labels_covered_points = Y_arr[indexes_points_covered]   # get a list only of Y labels of th
    mask = labels_covered_points == r.class_label
    result = np.array(indexes_points_covered)[mask]
    
    CORRECT_COVER_CACHE[repr(r)] = result, indexes_points_covered

    return result, indexes_points_covered
    
    
#%timeit get_correct_cover(r, df, Y)
#%timeit get_correct_cover_faster(r, df, Y)
#%timeit get_correct_cover_memoized(r, df, Y)

get_correct_cover_memoized(r, df, Y)

(array([96], dtype=int64),
 [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  13,
  14,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  47,
  51,
  53,
  54,
  71,
  74,
  79,
  80,
  83,
  88,
  96])

In [12]:



repr(r)
repr(list_of_rules[1])

NameError: name 'r' is not defined

In [25]:
def get_incorrect_cover(r, df, Y):
    correct_cover, full_cover = r.get_correct_cover(df, Y)
    return (sorted(list(set(full_cover) - set(correct_cover))))


def get_incorrect_cover_faster(r, df, Y):
    correct_cover, full_cover = r.get_correct_cover(df, Y)

    mask = full_cover == correct_cover
    
    return np.array(full_cover)[~mask]


all(get_incorrect_cover_faster(r, df, Y) == get_incorrect_cover(r, df, Y))




11.6 µs ± 77.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
20.9 µs ± 581 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [15]:
OVERLAP_CACHE = {}

for r1 in list_of_rules:
    for r2 in list_of_rules:
        OVERLAP_CACHE[repr(r1) + repr(r2)] = overlap(r1, r2, df)
        
        
len(OVERLAP_CACHE)

6561